In [ ]:
using DataFrames, CSV
using Random, Distributions, StatsBase
using Plots
using MLDataUtils, Clustering
using Distributed, SharedArrays

In [ ]:
@everywhere begin
    if !("src/" in LOAD_PATH)
        push!(LOAD_PATH, "src/")
    end    
end 
@everywhere begin
    if !("test/" in LOAD_PATH)
        push!(LOAD_PATH, "test/")
    end
end

In [ ]:
using TimerOutputs: @timeit, get_timer

using  Trees,  bound,  parallel,  Nodes
using  opt_func,  ub_func,  lb_func,  bb_func,  data_process
using groups

In [ ]:
D = 2 # 2 # 
LB_method = "CF+MILP+SG" 
seed = 1 # 
scheme = "sl"
dataname = "toy"
if dataname == "toy"
    nclst = 3 # parse(Int, ARGS[6]) 
    clst_n = 50 #parse(Int, ARGS[7])
    d = 2 # parse(Int, ARGS[8])
elseif dataname == "iris"
    datapackage = "datasets"
else
    datapackage = "nothing" # ARGS[6] # 
end

if scheme == "par"
    using MPI
    parallel.init()
end

In [ ]:
data = read_data("small_toy", "nothing")

In [ ]:
parallel.create_world()
parallel.root_println("Running $(parallel.nprocs()) processes.")
parallel.root_println("Start training $dataname with seed $seed.")

In [ ]:
if dataname == "toy"
    data, lbl, K = read_data(dataname, clst_n=clst_n, nclst=nclst, d=d)
else
    data, lbl, K = read_data(dataname; datapackage=datapackage)
end

In [ ]:
println(size(data))
Random.seed!(seed)
train, valid, test = stratifiedobs((view(data,:,:), lbl), (0.5, 0.25))
tr_x, tr_y = train
va_x, va_y = valid
te_x, te_y = test
lbl = Int.(lbl)
parallel.root_println(size(tr_x))
parallel.root_println(size(va_x))
parallel.root_println(size(te_x))
L_hat = maximum(counts(lbl))/length(lbl)
alp = 0.05 #alpha # parameter of complexity of the tree

X = Matrix(hcat(tr_x, va_x)) # Matrix(tr_x) # Matrix(data) # 
y = Vector(vcat(tr_y, va_y)) # Vector(tr_y) # lbl # 
y = Int.(y)
Y_g = opt_func.label_bin(y, K, "glb")
Y = Y_g # setup global Y for label binarization
Y_d = opt_func.label_bin(y, K, "dim")
p,n = size(X)
println("dimension: $p, class: $K")
time_w = @elapsed tree_w, objv_w = CART_base(X, Y_g, K, D, alp, L_hat)
println("cart cost: $objv_w")

In [ ]:
K = parallel.bcast(K)
alp = parallel.bcast(alp)
L_hat = parallel.bcast(L_hat)
objv_w = parallel.bcast(objv_w)
tree_w = parallel.bcast(tree_w)

In [ ]:
tree_plot(tree_w, "tree_w.png")

In [ ]:
# branch_bound(X, Y_g, K, D, tree_w, objv_w, alp, L_hat, LB_method, true, false)
X, Y, K, D, warm_start, UB_init, alpha, L_hat, method , prob , obbt = X, Y_g, K, D, tree_w, objv_w, alp, L_hat, LB_method, true, false

val=0
time_lapse::Int64=14400

In [ ]:
p, n_all = size(X);
alpha_s = alpha/n_all;
sortX = sort(X, dims=2) # sorted on each feature used in lb_func
eps = vec(mapslices(opt_func.mini_dist, sortX, dims=2)) # eps used in opt_func

In [ ]:
p = parallel.bcast(p)
n_all = parallel.bcast(n_all)
alpha_s = parallel.bcast(alpha_s)
eps = parallel.bcast(eps)

In [ ]:
UB = UB_init;
max_LB = 1e15; # used to save the best lower bound at the end (smallest but within the mingap)


In [ ]:
~, ~, X_proc, Y_proc, node, tree = groups.proc_data_preparation(X, Y, p, n_all, K, D, warm_start, method, false, 0, val)